#**Step 1: installazione delle librerie**

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00


#**Step 2: caricamento delle librerie**

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pandas as pd
from datasets import Dataset
import re
import random

import gc
import warnings
import ast


In [3]:
warnings.filterwarnings("ignore", category=UserWarning, message="torch.utils.checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, message="None of the inputs have requires_grad=True")

#**Step 3: collegamento a drive**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# change this to match your path
data_path_models = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 3a/Models"
data_path_data = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 3a/Datasets"

Mounted at /content/drive


#**Step 3: impostazione delle variabili**

In [5]:
model_name = "swap-uniba/LLaMAntino-2-7b-hf-ITA"
fine_tuned_model_name = "llamantino2_7b_TCO_it_2epoch"
testset_name = "splitted_test"

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4: generazione**

In [13]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text_for_sentence_aggregation(triple, sentence_len):
  max_length = len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] [/ANW]") + sentence_len * 1.2

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=int(max_length), temperature=0.1)
  result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

  if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
      generated_text = result[0]['generated_text']

      # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
      match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

      if match:
          content_within_anw_tag = match.group(1)
          text = content_within_anw_tag
      else:
          text = generated_text

  else:
      text = "Il risultato della pipeline è nullo."

  return text

def generation_for_sentence_aggregation(testset, testset_name):
  current_id = -1
  rows = []
  row = {}
  for i in range(len(testset)):
    if (testset['id'][i] > 390 and testset['id'][i] < 720): #testset['id'][i] < 1010
      id = testset['id'][i]
      triples = testset['triples'][i]
      data_unit = testset['data_unit'][i]
      shape = testset['shape'][i]
      shape_type = testset['shape_type'][i]
      local_size = testset['local_size'][i]
      size = testset['size'][i]

      if current_id == -1 or current_id != id:
        if current_id != -1:
          rows.append(row)

        row = {
            'id' : id,
            'data_unit' : data_unit,
            'sentences': [],
            'shape': shape,
            'shape_type': shape_type,
            'size': size,
        }

      prediction = generate_text_for_sentence_aggregation(triples, len(triples))
      row['sentences'].append(prediction)
      current_id = id

      print(f"======== Step {i} ========")
      print(f"tripla: {triples}")
      print(f"predizione: {prediction}")
      print('\n')

      # Crea un DataFrame da rows
      df = pd.DataFrame(rows)
      #df.to_csv(f'{data_path_gem_models}/{new_model}_decoding_to_aggregate.csv', index=False)
      df.to_csv(f'{data_path_models}/{testset_name}_step1.csv', index=False)

  # Crea un DataFrame da rows
  df = pd.DataFrame(rows)
  #df.to_csv(f'{data_path_gem_models}/{new_model}_decoding_to_aggregate.csv', index=False)
  df.to_csv(f'{data_path_models}/{testset_name}_step1.csv', index=False)


In [8]:
# caricamento di llama fine tuned
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, f'{data_path_models}/{fine_tuned_model_name}')
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [9]:
# caricamento del testset
testset = pd.read_csv(f'{data_path_data}/{testset_name}.csv')
testset.head()

,id,triples,data_unit,shape,shape_type,local_size,size
0,0,11th_Mississippi_Infantry_Monument category Co...,11th_Mississippi_Infantry_Monument category Co...,(X (X) (X) (X (X))),mixed,3,4
1,0,"Adams_County,_Pennsylvania hasToItsWest Frankl...",11th_Mississippi_Infantry_Monument category Co...,(X (X) (X) (X (X))),mixed,1,4
2,1,Bananaman broadcastedBy BBC Bananaman starring...,Bananaman broadcastedBy BBC | BBC city Broadca...,(X (X) (X (X))),mixed,2,3
3,1,BBC city Broadcasting_House,Bananaman broadcastedBy BBC | BBC city Broadca...,(X (X) (X (X))),mixed,1,3
4,2,Karlsruhe postalCode 76131–76229,Karlsruhe postalCode 76131–76229 |,(X (X)),NaN,1,1


In [14]:
# generazione delle frasi
generation_for_sentence_aggregation(testset, testset_name)

======== Step 634 ========
tripla: Liselotte_Grschebina birthPlace Karlsruhe Liselotte_Grschebina deathPlace Israel 
predizione: Liselotte Grschebina è nata a Karlsruhe ed è morta in Israele.


======== Step 635 ========
tripla: Israel ethnicGroup Arab_citizens_of_Israel 
predizione: Gli arabi sono un gruppo etnico israeliano.


======== Step 636 ========
tripla: Nord_(Year_of_No_Light_album) followedBy Live_at_Roadburn_2008_(Year_of_No_Light_album) Nord_(Year_of_No_Light_album) producer Year_of_No_Light 
predizione: L'album Nord (Year of No Light) è stato prodotto da Year of No Light e ha avuto come seguito Live at Roadburn 2008.


======== Step 637 ========
tripla: 1147_Stavropolis apoapsis 418476000000.0 1147_Stavropolis discovered 1929-06-11 
predizione: 1147 Stavropolis, che ha un apoapside di 418476000000.0, è stata scoperta il 1929-06-11.


======== Step 638 ========
tripla: Pontiac_Rageous bodyStyle Coupe Pontiac_Rageous manufacturer Pontiac Pontiac_Rageous productionEndYear 19

#**Step 4: Sentence Aggregation**

In [14]:
def generate_sentence_aggregation(sentences):
  text = "instruction=\"Devi unire insieme le seguenti frasi presenti in Input e parafrasarle in modo da rendere la frase prodotta più fluente."
  text += " Input:"
  text += " texts="
  text += " " + sentences
  text += " Output:"

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(text), temperature=0.1)
  result = pipe(f"{text}")
  text = result[0]['generated_text']

  match = re.search(r'Output: "(.*?)"', text)
  if match:
    prediction = match.group(1)
  else:
    prediction = text

  return prediction

def sentence_aggregation(testset, testset_name):
  rows = []
  for i in range(len(testset)):
    sentences = ast.literal_eval(testset['sentences'][i])
    sentences_text = ''
    j = 1

    if len(sentences) > 1:
      for sentence in sentences:
        sentences_text += f"Text {j}: \"{sentence}\"\n"
        j = j + 1
      prediction = generate_sentence_aggregation(sentences_text)
    else:
      sentences_text += f"Text {j}: \"{sentences[0]}\"\n"
      prediction = " " . join(sentences)

    print(f"===== STEP {i}")
    print(f"sentences_text: {sentences_text}")
    print(f"prediction: {prediction}")
    print('\n')

    row = {
        'data_unit' : testset['data_unit'][i],
        'predictions' : prediction,
    }

    rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(f'{data_path_models}/{testset_name}_step2.csv', index=False)

  df = pd.DataFrame(rows)
  df.to_csv(f'{data_path_models}/{testset_name}_step2.csv', index=False)


In [ ]:
# pulizia
del base_model
del model
del tokenizer
gc.collect()
gc.collect()

0

In [ ]:
gc.collect()
gc.collect()

0

In [7]:
# caricamento di llama
bnb_config = BitsAndBytesConfig(
  load_in_4bit= True,
  bnb_4bit_quant_type= "nf4",
  bnb_4bit_compute_dtype= torch.bfloat16,
  bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

(True, True)

In [8]:
# caricamento del test set
testset = pd.read_csv(f'{data_path_models}/{testset_name}_step1.csv')
testset.head()

,id,data_unit,sentences,shape,shape_type,size
0,0,11th_Mississippi_Infantry_Monument category Co...,"[""Il monumento dell'11° Fanteria del Mississip...",(X (X) (X) (X (X))),mixed,4
1,1,Bananaman broadcastedBy BBC | BBC city Broadca...,['Bananaman è stato trasmesso dalla BBC e inte...,(X (X) (X (X))),mixed,3
2,2,Karlsruhe postalCode 76131–76229 |,['Il codice postale di Karlsruhe è 76131-76229.'],(X (X)),NaN,1
3,3,Turkish_people religion Irreligion |,"[""La religione dei turchi è l'irreligione.""]",(X (X)),NaN,1
4,4,ENAIRE city Madrid | Adolfo_Suárez_Madrid–Bara...,"['ENAIRE si trova nella città di Madrid.', ""L'...",(X (X) (X) (X) (X (X))),mixed,5


In [15]:
# generazione delle frasi
sentence_aggregation(testset, f'{testset_name}_prompt1')

Output streaming troncato alle ultime 5000 righe.
===== STEP 269
sentences_text: Text 1: "Akeem Ayers ha debuttato con i Tennessee Titans e ha una scelta al numero 39."

prediction: Akeem Ayers ha debuttato con i Tennessee Titans e ha una scelta al numero 39.


===== STEP 270
sentences_text: Text 1: "Olga Bondareva è nata il 27 aprile 1937 ed è stata chiamata Olga Nikolaevna Bondareva. Ha frequentato l'Università Statale di Leningrado."
Text 2: "Olga Bondareva, nata a Leningrado, nell'Unione Sovietica, è nota per avere contribuito alla Bondareva-Shapley theorem."

prediction: Olga Bondareva è nata il 27 aprile 1937 ed è stata chiamata Olga Nikolaevna Bondareva. Ha frequentato l'Università Statale di Leningrado. Olga Bondareva, nata a Leningrado, nell'Unione Sovietica, è nota per avere contribuito alla Bondareva-Shapley theorem.


===== STEP 271
sentences_text: Text 1: "L'Acharya Institute of Technology si trova a Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta M